In [2]:
import boto3
import pandas as pd
import json
import time
import requests
import zipfile
from io import BytesIO

In [3]:
# AWS credentials
with open("../resources/credentials.json", "r") as f:
    creds = json.load(f)
    
ACCESS_KEY = creds["Access key ID"]
SECRET_KEY = creds["Secret access key"]
region = "eu-west-1"

# Clients
lex_runtime = "lex-runtime"
lex_models = "lex-models"

# Bot details
with open("../resources/bot_config.json", "r") as f:
    bot_config = json.load(f)
    
bot_name = bot_config["bot_name"]
bot_alias = bot_config["bot_alias"]

In [4]:
client = boto3.client(lex_models,
                      aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY,
                      region_name=region)

In [ ]:
response = client.create_bot_version(
    name=bot_name,
)
bot_version = response["version"]
bot_version

In [ ]:
# Export Bot
response = client.get_export(
    name=bot_name,
    version=bot_version,
    resourceType='BOT',
    exportType='LEX'
)

response

In [ ]:
url = response['url']
file_out = url.split('/')[3].split("?")[0]

with open(f"../bots/{file_out}", 'wb') as f:
    
    f.write(requests.get(url).content)

In [7]:
file_out = "banking_seventy_seven_1_f96d1de8-b68c-4732-82e4-0290663024e3_Bot_LEX_V1.zip"
with zipfile.ZipFile(f"../bots/{file_out}", "r") as bot_zip:
    
    fname = bot_zip.namelist()[0]
    with bot_zip.open(fname) as f:
        bot_extract = json.loads(f.read().decode("utf-8"), )
        

In [ ]:
# Delete current bot and existing intents

# Check for aliases
response = client.get_bot_aliases(
    botName=bot_name,
)

if len(response['BotAliases']) > 0:
    for alias in response['BotAliases']:
        alias_name = alias["name"]
        
        print(f"Deleting alias: {alias_name}", end=" - ")
        response = client.delete_bot_alias(
            name=alias["name"],
            botName=bot_name
        )
        time.sleep(1)
        print("Done")

# Delete bot
print(f"\nDeleting Bot: {bot_name}", end=" - ")
response = client.delete_bot(
    name=bot_name
)
time.sleep(1)
print("Done.\n")

In [8]:
# Get existing intents
existing_intents = [intent["name"] for intent in bot_extract["resource"]["intents"]]


# Delete intents
for i, intent in enumerate(existing_intents):
    print(f"\rDeleting intent ({i+1} of {len(existing_intents)}): {intent}", end=" - ")
    not_deleted = True
    while not_deleted:
        try:
            response = client.delete_intent(
                name=intent
            )
            not_deleted = False
        except:
            time.sleep(1)
            
    print("Done.", end="")

In [ ]:
# # Delete all intents
all_intents = [i['name'] for i in client.get_intents(maxResults=50)['intents']]

# Delete all intents
for intent in all_intents:
    print(f"Deleting intent: {intent}", end=" - ")
    
    try:
        client.delete_intent(
            name=intent
        )
        time.sleep(1)
    except:
        time.sleep(1)
    print("Done.")

# Update bit with new utterances

In [5]:
# Read training intents/utterances
# df = pd.read_excel("../data/train.xlsx")
df = pd.read_excel("../data/train_tiny.xlsx")
# df["intent"] = df["intent"].str.replace('[^\w\s]','', regex=True)
# df["text"] = df["text"].str.replace('[^\w\s]','', regex=True).str.strip().str.strip("_")

# df = df.drop_duplicates(subset=["text", "intent"]).reset_index(drop=True)

df.head(2)

,text,label,intent
0,I cannot view the refunded amount in my account,51,Refund_not_showing_up
1,My refund is not showing up in my account Wher...,51,Refund_not_showing_up


In [9]:
intents_out = []
for intent in df['intent'].unique():
    
    utters = df.loc[df['intent'] == intent, "text"].values.tolist()
    
    if intent in existing_intents:
        
        intent_out = list(filter(lambda d: d['name'] == intent, 
                                 bot_extract["resource"]["intents"]))[0].copy()
        
        intent_out['sampleUtterances'] = utters
        intent_out['version'] = "$LATEST"
        
    else:
        
        intent_out = {
            'name': intent,
            'version': "$LATEST",
            'fulfillmentActivity': {'type': 'ReturnIntent'},
            'sampleUtterances': utters,
            'slots': []
            }
    intents_out.append(intent_out)
    

In [10]:
# Update bot with new utterances
bot_extract["resource"]["intents"] = intents_out
mem_zip = BytesIO()

# Zip bot
with zipfile.ZipFile(mem_zip, "w") as z:
    with z.open(f"{bot_name}.json", "w") as c:
        c.write(json.dumps(bot_extract, indent=4).encode("utf-8"))
        
# Zip bot
with zipfile.ZipFile(f"../bots/{bot_name}_update.zip", "w") as z:
    with z.open(f"{bot_name}.json", "w") as c:
        c.write(json.dumps(bot_extract, indent=4).encode("utf-8"))
  
       
response = client.start_import(
    payload=mem_zip.getvalue(),
    resourceType='BOT',
    mergeStrategy='OVERWRITE_LATEST'
)

In [22]:
client.get_import(
    importId=response["importId"])

{'ResponseMetadata': {'RequestId': 'c58b1539-dc0d-402e-b439-6a9c82f8a81b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Thu, 24 Feb 2022 13:21:52 GMT',
   'x-amzn-requestid': 'c58b1539-dc0d-402e-b439-6a9c82f8a81b',
   'content-length': '215',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'name': 'banking_seventy_seven',
 'resourceType': 'BOT',
 'mergeStrategy': 'OVERWRITE_LATEST',
 'importId': '90642d98-8aaf-4fe8-a48c-5734916b0f5b',
 'importStatus': 'COMPLETE',
 'createdDate': datetime.datetime(2022, 2, 24, 15, 21, 45, 650000, tzinfo=tzlocal())}